In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RF

from sklearn.metrics import r2_score as r2

In [2]:
data = pd.read_csv('D:\Учебное\GeekBrains\Python_ds\Project\_train.csv', sep=';')

In [3]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [4]:
def correct(_data):
    
    _data.loc[_data['Square'] < _data['LifeSquare'], ['LifeSquare', 'Square']] = _data.loc[_data['Square'] < _data['LifeSquare'], ['Square', 'LifeSquare']].to_numpy()
    _data.loc[_data['Square'] < 15 , ['Square']] = _data.loc[_data['Square'] < 15 , ['Square']].apply(lambda x: x * 10)
    _data.loc[_data['LifeSquare'] < 15 , ['LifeSquare']] = _data.loc[_data['LifeSquare'] < 15 , ['LifeSquare']].apply(lambda x: x * 10)
    _data.loc[_data['LifeSquare'] / _data['Square'] < 0.4, ['LifeSquare']] = _data.loc[_data['LifeSquare']/_data['Square'] < 0.4, ['Square']].apply(lambda x: x * 0.6)
    _data['LifeSquare'] = _data['LifeSquare'].fillna(float(1))
    _data = _data.assign(MeanRooms = round(_data.LifeSquare / 21))
    _data.loc[_data['Rooms'] > 5 , ['Rooms' , 'MeanRooms']] = _data.loc[_data['Rooms'] > 5 , ['MeanRooms' , 'Rooms']].to_numpy()
    _data.loc[_data['Rooms'] < 1 , ['Rooms' , 'MeanRooms']] = _data.loc[_data['Rooms'] < 1 , ['MeanRooms' , 'Rooms']].to_numpy()
    
    return _data

In [5]:
def complete(train):
    
    train = pd.merge(train, mn_sq, on=['Rooms'], how='left')
    train = pd.merge(train, mean_price, on=['DistrictId' , 'Rooms'], how='left')
    train = train.assign(MeanSquare2 = data.LifeSquare / data.Rooms)
    
    return train

In [6]:
def correctSquare(data):
    data = data.assign(MeanSquare3 = data.MeanSquare2 * data.Rooms)
    data.loc[data['Square'] > 250, ['Square' , 'MeanSquare3']] = data.loc[data['Square'] > 250, ['MeanSquare3' , 'Square']].to_numpy()
    
    return data

In [7]:
def add_distr_size(df, distr_size):
    df = pd.merge(df, distr_size, on='DistrictId', how='left')
    df['large_district'] = df['large_district'].fillna(0)
    return df

In [8]:
data = correct(data)
valid = correct(valid)
train = correct(train)


In [9]:
mn_sq = data.groupby(['Rooms'], as_index=False)[['LifeSquare']].mean().rename(columns={'LifeSquare': 'MeanSquare1'})
data = data.assign(Meanprice = data.Price / data.Square)
mean_price = data.groupby(['DistrictId' , 'Rooms'], as_index=False)[['Meanprice']].mean()
distr_size = (data['DistrictId'].value_counts() > 180).astype(int).reset_index()
distr_size.columns = ['DistrictId', 'large_district']

In [10]:
valid = complete(valid)
train = complete(train)

In [11]:
valid = correctSquare(valid)
train = correctSquare(train)

In [12]:
train = add_distr_size(train, distr_size)
valid = add_distr_size(valid, distr_size)

In [13]:
model = RF(n_estimators=20, max_depth=6, random_state=42)

In [14]:
feats = ['Rooms', 
         'Square',
         'MeanSquare1',
         'DistrictId',
         'Meanprice',
         'Ecology_1',
         'large_district',
         'Shops_1'
        ]

In [15]:
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
pred_train = model.predict(train.loc[:, feats])
pred_valid = model.predict(valid.loc[:, feats])

In [17]:
r2(train['Price'], pred_train), r2(valid['Price'], pred_valid)

(0.7905830703234169, 0.7432895710239746)

In [18]:
Meanprice = train["Meanprice"].mean()

### Test

In [19]:
test = pd.read_csv('D:\Учебное\GeekBrains\Python_ds\Project\_test.csv', sep=';')

In [20]:
test = correct(test)

In [21]:
test = complete(test)

In [22]:
test = correctSquare(test)

In [23]:
test = add_distr_size(test, distr_size)

In [24]:
test['Meanprice'] = test['Meanprice'].fillna(Meanprice)

In [25]:
test['Price'] = model.predict(test.loc[:, feats])

In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 26 columns):
Id                5000 non-null int64
DistrictId        5000 non-null int64
Rooms             5000 non-null float64
Square            5000 non-null float64
LifeSquare        5000 non-null float64
KitchenSquare     5000 non-null float64
Floor             5000 non-null int64
HouseFloor        5000 non-null float64
HouseYear         5000 non-null int64
Ecology_1         5000 non-null float64
Ecology_2         5000 non-null object
Ecology_3         5000 non-null object
Social_1          5000 non-null int64
Social_2          5000 non-null int64
Social_3          5000 non-null int64
Healthcare_1      2623 non-null float64
Helthcare_2       5000 non-null int64
Shops_1           5000 non-null int64
Shops_2           5000 non-null object
MeanRooms         5000 non-null float64
MeanSquare1       5000 non-null float64
Meanprice         5000 non-null float64
MeanSquare2       5000 non-null fl

In [27]:
test.loc[:, ['Id', 'Price']].to_csv('D:\Учебное\GeekBrains\Python_ds\Project\LVoronov_predictions.csv', index=None, sep=',')